In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTENC, ADASYN

Os dados contidos na planilha 'bureau_balance' demonstram o status de cada empréstimo mês a mês para todos os clientes. Vamos verificar como se comportam os dados:

In [2]:
df_bureau_balance = pd.read_csv('bureau_balance.csv')

In [3]:
df_bureau_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   SK_ID_BUREAU    int64 
 1   MONTHS_BALANCE  int64 
 2   STATUS          object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


In [4]:
df_bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [5]:
df_bureau_balance['STATUS'].value_counts()

C    13646993
0     7499507
X     5810482
1      242347
5       62406
2       23419
3        8924
4        5847
Name: STATUS, dtype: int64

Percebe-se que há uma certa quantidade de dados 'missing' (os dados de status marcados como 'X'). Como estes estão distribuidos entre os dados preenchidos sem nenhum padrão, muitas vezes não havendo nenhum dado preenchido para certos ID's, é difícil intuir uma forma precisa de preenchimento. Como os dados ausentes representam menos de 20% da base total, estes serão considerados apenas como 0 ou C, que representam meses com pagamento em dia. Como o intuito desta parte da análise é verificar se em algum dos meses houve inadimplencia, qualquer mês que seja, não haverá tanto impacto se a consideração for feita desta forma.

Será criado um DF auxiliar, que irá receber um valor unitário para cada mês de inadimplencia do empréstimo (coluna AUX). Esse dataframe será então agrupado pelos ID's, e todos os empréstimos que estiverem com a contagem maior do que 0 (representando que houve inadimplência em algum mês) receberão valor 1 na nova coluna 'DEFAULT', enquanto que os demais receberão valor 0. Essa nova coluna vai indicar se há histórico de inadimplência para cada empréstimo, sendo 0 = não e 1 = sim.

In [6]:
df_aux=df_bureau_balance.copy()
df_aux['AUX']=np.where(((df_aux['STATUS'] == '1') | (df_aux['STATUS'] == '2') | (df_aux['STATUS'] == '3') | (df_aux['STATUS'] == '4') | (df_aux['STATUS'] == '5')), 1, 0)
df_aux = df_aux.groupby(['SK_ID_BUREAU']).sum().drop(['MONTHS_BALANCE'],axis='columns')
df_aux['DEFAULT']=np.where(df_aux['AUX']>=1,1,0)
df_aux.head(10)

,AUX,DEFAULT
SK_ID_BUREAU,,
5001709,0,0
5001710,0,0
5001711,0,0
5001712,0,0
5001713,0,0
5001714,0,0
5001715,0,0
5001716,0,0
5001717,0,0


Vamos confrontar os dados com o dataframe original e verificar se os ID's sinalizados com default 1 realmente possuem histórico de inadimplência:

In [7]:
df_bureau_balance[df_bureau_balance['SK_ID_BUREAU']==5001717]['STATUS'].value_counts()

0    17
C     5
Name: STATUS, dtype: int64

In [8]:
df_bureau_balance[df_bureau_balance['SK_ID_BUREAU']==5001718]['STATUS'].value_counts()

0    24
X    10
C     3
1     2
Name: STATUS, dtype: int64

Percebe-se que o algoritmo funciona corretamente, com os empréstimos com algum histórico de inadimplência sendo marcados com DEFAULT = 1. Vamos então dropar a coluna auxiliar e esta parte da análise está pronta.

In [9]:
df_aux.drop('AUX', axis='columns', inplace=True)

In [10]:
df_aux['DEFAULT'].value_counts()

0    714131
1    103264
Name: DEFAULT, dtype: int64

Passemos então ao DF bureau:

In [11]:
df_bureau = pd.read_csv('bureau.csv')
df_bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


In [12]:
df_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


O dataframe criado anteriormente pode ser agrupado com o da base 'bureau' através de um join com a coluna 'SK_ID_BUREAU', comum às duas bases. Há mais empréstimos na base de dados bureau do que na base tratada, então alguns valores ficarão missing. Façamos como Left join para passar todos os dados de 'DEFAULT' para os dados existentes na base bureau.

In [13]:
df_bureau_aux = df_bureau.join(df_aux, on='SK_ID_BUREAU', how='left')

In [14]:
df_bureau_aux.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DEFAULT
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN


In [15]:
df_bureau_aux.isnull().sum()

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
DEFAULT                    942074
dtype: int64

Antes de dividir as bases, vamos verificar se há alguma coluna irrelevante ou dados missing a serem dropados. Percebe-se que há dados nulos nas colunas 6,7,8,10,11,12 e 16. Vamos estudar caso a caso como podemos tratar estes dados. Vejamos o que cada feature representa:

    * DAYS_CREDIT_ENDDATE: Prazo restante do empréstimo em questão no dia em que foi feita a aplicação para o novo crédito;
    * DAYS_ENDDATE_FACT: Dias desde o fechamento do empréstimo em questão na data em que foi feita a aplicação para o novo crédito (apenas para empréstimos finalizados);
    * AMT_CREDIT_MAX_OVERDUE: Tempo máximo de atraso até a data da aplicação para o novo crédito;
    * AMT_CREDIT_SUM: Total de crédito atual
    * AMT_CREDIT_SUM_DEBT: Débito atual
    * AMT_CREDIT_SUM_LIMIT: Limite do cartão de crédito
    * AMT_ANNUITY: Anuidade do crédito

In [16]:
df_bureau_aux[df_bureau_aux['DAYS_CREDIT_ENDDATE'].isna()].head(10)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DEFAULT
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN
17,238881,5714486,Active,currency 1,-381,0,NaN,NaN,NaN,0,450000.0,520920.0,NaN,0.0,Consumer credit,-4,NaN,NaN
20,238881,5714489,Active,currency 1,-392,0,NaN,NaN,0.0,0,252000.0,23679.0,228320.100,0.0,Credit card,-22,NaN,NaN
32,136226,5714503,Closed,currency 1,-559,0,NaN,-355.0,0.0,0,110250.0,0.0,0.000,0.0,Credit card,-351,NaN,NaN
43,435112,5714518,Active,currency 1,-1483,0,NaN,NaN,NaN,0,200250.0,0.0,NaN,0.0,Credit card,-848,NaN,NaN
75,303740,5714554,Active,currency 1,-581,0,NaN,NaN,0.0,0,384750.0,263056.5,121690.170,0.0,Credit card,-22,NaN,NaN
97,119939,5714580,Active,currency 1,-2401,0,NaN,NaN,0.0,0,99000.0,45036.0,53962.695,0.0,Credit card,-22,NaN,NaN
108,419892,5714596,Active,currency 1,-381,0,NaN,NaN,NaN,0,2767500.0,2739622.5,NaN,0.0,Mortgage,-27,NaN,NaN
136,338464,5714634,Active,currency 1,-140,0,NaN,NaN,0.0,0,144000.0,143019.0,980.280,0.0,Credit card,-22,NaN,NaN
148,444864,5714647,Active,currency 1,-286,0,NaN,NaN,0.0,0,364500.0,0.0,364500.000,0.0,Credit card,-29,NaN,NaN


Podemos tentar prever os dados missing da coluna Default utilizando um modelo de machine learning com os dados preenchidos. 

In [17]:
df_bureau_predict = df_bureau_aux[df_bureau_aux['DEFAULT'].isnull()]
df_bureau_model = df_bureau_aux[df_bureau_aux['DEFAULT'].notnull()]

Usaremos a base 'model' para construirmos nosso modelo. A base predict ficará salva para aplicação do modelo posteriormente

Vamos analisar agora as bases de dados 'Credit_Card_Balance' e 'Installments_Payments', que se conectam através da feature 'SK_ID_PREV'

### INSTALLMENTS PAYMENTS

Analisemos a base de parcelamentos. Aqui são descritas cada parcela paga (ou não) para cada empréstimo passado de cada cliente:

In [18]:
df_installments_payments = pd.read_csv('installments_payments.csv')

In [19]:
df_installments_payments[df_installments_payments['NUM_INSTALMENT_VERSION']==0.0].count()

SK_ID_PREV                4082498
SK_ID_CURR                4082498
NUM_INSTALMENT_VERSION    4082498
NUM_INSTALMENT_NUMBER     4082498
DAYS_INSTALMENT           4082498
DAYS_ENTRY_PAYMENT        4080791
AMT_INSTALMENT            4082498
AMT_PAYMENT               4080791
dtype: int64

In [20]:
df_installments_payments.groupby(['SK_ID_PREV','SK_ID_CURR','NUM_INSTALMENT_VERSION'], as_index = False, sort = False).sum()[df_installments_payments['NUM_INSTALMENT_VERSION']==0.0].count()

C:\Users\leokm\AppData\Local\Temp/ipykernel_7864/2987370520.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_installments_payments.groupby(['SK_ID_PREV','SK_ID_CURR','NUM_INSTALMENT_VERSION'], as_index = False, sort = False).sum()[df_installments_payments['NUM_INSTALMENT_VERSION']==0.0].count()


SK_ID_PREV                467968
SK_ID_CURR                467968
NUM_INSTALMENT_VERSION    467968
NUM_INSTALMENT_NUMBER     467968
DAYS_INSTALMENT           467968
DAYS_ENTRY_PAYMENT        467968
AMT_INSTALMENT            467968
AMT_PAYMENT               467968
dtype: int64

In [21]:
df_installments_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   SK_ID_CURR              int64  
 2   NUM_INSTALMENT_VERSION  float64
 3   NUM_INSTALMENT_NUMBER   int64  
 4   DAYS_INSTALMENT         float64
 5   DAYS_ENTRY_PAYMENT      float64
 6   AMT_INSTALMENT          float64
 7   AMT_PAYMENT             float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


In [22]:
df_installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [23]:
df_installments_payments.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

Percebe-se que há uma pequena quantidade de nulos. Vejamos como se comportam:

In [24]:
df_installments_payments[df_installments_payments['DAYS_ENTRY_PAYMENT'].isna()].head(10)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
3764207,1531600,103793,1.0,7,-668.0,NaN,49741.020,NaN
3764208,1947105,159974,1.0,24,-36.0,NaN,22849.515,NaN
3764209,1843773,167270,1.0,22,-20.0,NaN,48092.355,NaN
3764210,1691592,192536,1.0,5,-2561.0,NaN,7675.425,NaN
3764211,1531299,157088,0.0,11,-1847.0,NaN,67.500,NaN
3764212,1562727,187345,0.0,23,-5.0,NaN,11401.740,NaN
3764213,1265332,167942,1.0,5,-14.0,NaN,11650.500,NaN
3764214,1531600,103793,1.0,25,-128.0,NaN,49741.020,NaN
3764215,1852469,103171,7.0,3,-49.0,NaN,311738.355,NaN
3764216,1054684,142217,0.0,38,-1469.0,NaN,67.500,NaN


Por representarem uma pequena parcela do dataset, estes dados poderiam ser dropados em outras ocasiões. Aqui, entretanto, os dados missing representam parcelas não pagas. Uma vez que buscamos históricos de inadimplência, estes dados são úteis para a construção de nosso modelo. Desta forma, em ambas as features ('DAYS_ENTRY_PAYMENT' - Dias desde o pagamento da parcela / 'AMT_PAYMENT' - Total pago) serão preenchidas com zeros, para utilização em fase posterior

In [25]:
df_installments_payments.fillna(0, inplace=True)
df_installments_payments.isna().sum()

SK_ID_PREV                0
SK_ID_CURR                0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_INSTALMENT           0
DAYS_ENTRY_PAYMENT        0
AMT_INSTALMENT            0
AMT_PAYMENT               0
dtype: int64

Será então criada uma nova feature para este DF resultante, 'DAYS_LATE', que representa o total de dias de atraso no pagamento para a parcela em questão (dias negativos representam pagamentos antes do prazo). Será criada também uma feature para representar a quantidade não paga para a parcela em questão, 'AMT_UNPAID'.

In [26]:
df_installments_payments['DAYS_BALANCE'] = df_installments_payments['DAYS_ENTRY_PAYMENT'] - df_installments_payments['DAYS_INSTALMENT']
df_installments_payments = df_installments_payments.groupby(['SK_ID_PREV','SK_ID_CURR','NUM_INSTALMENT_NUMBER'], 
    as_index = False, 
    sort = False).agg({'NUM_INSTALMENT_VERSION':'first','DAYS_INSTALMENT':'first','DAYS_ENTRY_PAYMENT':'first','DAYS_BALANCE':'sum','AMT_INSTALMENT':'first','AMT_PAYMENT':'sum'})
df_installments_payments['AMT_UNPAID'] = df_installments_payments['AMT_INSTALMENT'] - df_installments_payments['AMT_PAYMENT']
df_installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,DAYS_BALANCE,AMT_INSTALMENT,AMT_PAYMENT,AMT_UNPAID
0,1054186,161674,6,1.0,-1180.0,-1187.0,-7.0,6948.360,6948.360,0.000
1,1330831,151639,34,0.0,-2156.0,-2156.0,0.0,1716.525,1716.525,0.000
2,2085231,193053,1,2.0,-63.0,-63.0,23.0,25425.000,26056.035,-631.035
3,2452527,199697,3,1.0,-2418.0,-2426.0,-8.0,24350.130,24350.130,0.000
4,2714724,167756,2,1.0,-1383.0,-1366.0,-21.0,2165.040,2165.040,0.000


Neste ponto, as features 'NUM_INSTALMENT_VERSION' e 'NUM_INSTALMENT_NUMBER' (que correspondem à versão do calendário de pagamento das parcelas e ao número da parcela observada, respectivamente) passam a ter pouco valor analítico, então serão dropadas. As features 'DAYS_ENTRY_PAYMENT','DAYS_INSTALMENT' e 'AMT_PAYMENT' também serão dropadas, por fazerem parte das features resultantes criadas anteriormente.

In [27]:
df_installments_payments.drop(['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER','DAYS_ENTRY_PAYMENT','AMT_PAYMENT','DAYS_INSTALMENT'], axis = 'columns', inplace=True)
df_installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,DAYS_BALANCE,AMT_INSTALMENT,AMT_UNPAID
0,1054186,161674,-7.0,6948.360,0.000
1,1330831,151639,0.0,1716.525,0.000
2,2085231,193053,23.0,25425.000,-631.035
3,2452527,199697,-8.0,24350.130,0.000
4,2714724,167756,-21.0,2165.040,0.000


Por fim, os dados são agrupados pelos ID's de cada transação e cada cliente, formando um novo DF que conta com o balanço de dias em atraso (valores negativos significa um saldo de dias adiantados), total de todas as parcelas e total não pago:

In [28]:
df_installments_payments = df_installments_payments.groupby(['SK_ID_PREV','SK_ID_CURR'], as_index = False, sort = False).sum()
df_installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,DAYS_BALANCE,AMT_INSTALMENT,AMT_UNPAID
0,1054186,161674,-316.0,83367.180,0.000
1,1330831,151639,-246.0,239518.395,0.000
2,2085231,193053,16.0,33818.175,-631.035
3,2452527,199697,-48.0,145997.865,0.000
4,2714724,167756,-38.0,16186.410,0.000


### CREDIT CARD BALANCE

Neste ponto será analisada a base 'Credit Card Balance', que agrega dados do cartão de crédito de alguns clientes mês a mês

In [29]:
df_credit_card_balance = pd.read_csv('credit_card_balance.csv')

In [30]:
df_credit_card_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SK_ID_PREV                  int64  
 1   SK_ID_CURR                  int64  
 2   MONTHS_BALANCE              int64  
 3   AMT_BALANCE                 float64
 4   AMT_CREDIT_LIMIT_ACTUAL     int64  
 5   AMT_DRAWINGS_ATM_CURRENT    float64
 6   AMT_DRAWINGS_CURRENT        float64
 7   AMT_DRAWINGS_OTHER_CURRENT  float64
 8   AMT_DRAWINGS_POS_CURRENT    float64
 9   AMT_INST_MIN_REGULARITY     float64
 10  AMT_PAYMENT_CURRENT         float64
 11  AMT_PAYMENT_TOTAL_CURRENT   float64
 12  AMT_RECEIVABLE_PRINCIPAL    float64
 13  AMT_RECIVABLE               float64
 14  AMT_TOTAL_RECEIVABLE        float64
 15  CNT_DRAWINGS_ATM_CURRENT    float64
 16  CNT_DRAWINGS_CURRENT        int64  
 17  CNT_DRAWINGS_OTHER_CURRENT  float64
 18  CNT_DRAWINGS_POS_CURRENT    float64
 19  CNT_INSTALMENT_MATURE

In [31]:
df_credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [32]:
df_credit_card_balance.isna().sum()

SK_ID_PREV                         0
SK_ID_CURR                         0
MONTHS_BALANCE                     0
AMT_BALANCE                        0
AMT_CREDIT_LIMIT_ACTUAL            0
AMT_DRAWINGS_ATM_CURRENT      749816
AMT_DRAWINGS_CURRENT               0
AMT_DRAWINGS_OTHER_CURRENT    749816
AMT_DRAWINGS_POS_CURRENT      749816
AMT_INST_MIN_REGULARITY       305236
AMT_PAYMENT_CURRENT           767988
AMT_PAYMENT_TOTAL_CURRENT          0
AMT_RECEIVABLE_PRINCIPAL           0
AMT_RECIVABLE                      0
AMT_TOTAL_RECEIVABLE               0
CNT_DRAWINGS_ATM_CURRENT      749816
CNT_DRAWINGS_CURRENT               0
CNT_DRAWINGS_OTHER_CURRENT    749816
CNT_DRAWINGS_POS_CURRENT      749816
CNT_INSTALMENT_MATURE_CUM     305236
NAME_CONTRACT_STATUS               0
SK_DPD                             0
SK_DPD_DEF                         0
dtype: int64

Vamos verificar quantos registros de créditos há nesse dataset. Para isso, agruparemos os dados pelos ID's, onde será desconsiderado o efeito mês a mês:

In [33]:
df_credit_card_balance.groupby(['SK_ID_PREV','SK_ID_CURR'], as_index = False, sort = False).sum()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
0,2562384,378907,-1175,1.314752e+06,9036000,67500.0,124941.645,0.0,57441.645,92335.590,...,1.271879e+06,1.322192e+06,1.322192e+06,1.0,8,0.0,7.0,983.0,6,6
1,2582071,363914,-4656,3.158007e+06,6120000,293850.0,358313.670,0.0,64463.670,206433.000,...,3.003606e+06,3.160853e+06,3.160853e+06,31.0,42,0.0,11.0,4209.0,1,1
2,1740877,371185,-703,4.165194e+06,13999500,506250.0,699566.715,0.0,193316.715,214735.860,...,4.011338e+06,4.153952e+06,4.153952e+06,21.0,104,0.0,83.0,666.0,0,0
3,1389973,337855,-120,2.837609e+06,3015000,50850.0,259850.835,0.0,209000.835,127967.715,...,2.713100e+06,2.794484e+06,2.794484e+06,15.0,75,0.0,60.0,91.0,0,0
4,1891521,126868,-4656,1.867620e+07,21870000,763650.0,1372493.385,0.0,608843.385,970803.810,...,1.810148e+07,1.867442e+07,1.867442e+07,40.0,144,0.0,104.0,5136.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104302,2339982,318916,-1,0.000000e+00,45000,0.0,0.000,0.0,0.000,0.000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0.0,0.0,0.0,0,0
104303,2720102,174455,-1,0.000000e+00,225000,0.0,0.000,0.0,0.000,0.000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0.0,0.0,0.0,0,0
104304,1897864,315041,-2,0.000000e+00,270000,0.0,0.000,0.0,0.000,0.000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0.0,0.0,0.0,0,0
104305,2481982,382749,-1,0.000000e+00,270000,0.0,0.000,0.0,0.000,0.000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0.0,0.0,0.0,0,0


As features 'SD_DPD_DEF','CNT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_CURRENT','CNT_DRAWINGS_POS_CURRENTS','CNT_DRAWINGS_OTHER_CURRENT' serão dropadas em um primeiro momento por não serem significativas para a análise pretendida

In [34]:
df_credit_card_balance.drop(['CNT_DRAWINGS_POS_CURRENT','CNT_DRAWINGS_OTHER_CURRENT','CNT_DRAWINGS_CURRENT','CNT_DRAWINGS_ATM_CURRENT','SK_DPD_DEF'],axis='columns',inplace=True)
df_credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,35.0,Active,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,69.0,Active,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,30.0,Active,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,10.0,Active,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,101.0,Active,0


Agora analisemos os dados nulos:

Percebe-se que os casos em que os valores estão missing são para situações sem movimentação no cartão de crédito. Percebe-se também que algumas dessas situações sem movimentações permanece sem movimentação por todo o período registrado, o que pode indicar contas em desuso. Desta forma, os valores missing do DF serão substituídos por 0.

In [35]:
df_credit_card_balance.fillna(0, inplace=True)
df_credit_card_balance.isna().sum()

SK_ID_PREV                    0
SK_ID_CURR                    0
MONTHS_BALANCE                0
AMT_BALANCE                   0
AMT_CREDIT_LIMIT_ACTUAL       0
AMT_DRAWINGS_ATM_CURRENT      0
AMT_DRAWINGS_CURRENT          0
AMT_DRAWINGS_OTHER_CURRENT    0
AMT_DRAWINGS_POS_CURRENT      0
AMT_INST_MIN_REGULARITY       0
AMT_PAYMENT_CURRENT           0
AMT_PAYMENT_TOTAL_CURRENT     0
AMT_RECEIVABLE_PRINCIPAL      0
AMT_RECIVABLE                 0
AMT_TOTAL_RECEIVABLE          0
CNT_INSTALMENT_MATURE_CUM     0
NAME_CONTRACT_STATUS          0
SK_DPD                        0
dtype: int64

Vamos então agrupar os dados para manter apenas um registro para cada empréstimo. Iremos somar os totais pagos e os totais sacados, bem como os dias de atraso e a quantidade de parcelas pagas, enquanto o valor do limite de crédito será mantido o último valor.

In [36]:
df_credit_card_balance[df_credit_card_balance['SK_ID_PREV']==2562384].sort_values('MONTHS_BALANCE')

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD
1811769,2562384,378907,-48,69186.690,270000,67500.0,67500.000,0.0,0.000,0.000,0.000,0.000,67500.000,67841.190,67841.190,0.0,Active,0
2584127,2562384,378907,-47,62727.075,270000,0.0,0.000,0.0,0.000,3498.975,4500.000,4500.000,60979.410,63246.825,63246.825,1.0,Active,0
3284876,2562384,378907,-46,62577.900,270000,0.0,0.000,0.0,0.000,3399.795,3600.000,3600.000,60796.035,63082.080,63082.080,2.0,Active,0
2653597,2562384,378907,-45,61484.400,270000,0.0,0.000,0.0,0.000,3346.695,3600.000,3600.000,59734.170,61980.255,61980.255,3.0,Active,0
1039133,2562384,378907,-44,60736.680,270000,0.0,0.000,0.0,0.000,3291.390,3375.000,3375.000,59078.160,61224.345,61224.345,4.0,Active,0
2021610,2562384,378907,-43,59786.865,270000,0.0,0.000,0.0,0.000,3242.250,3375.000,3375.000,58095.090,60267.735,60267.735,5.0,Active,0
1764744,2562384,378907,-42,58743.675,270000,0.0,0.000,0.0,0.000,3194.190,3375.000,3375.000,57133.755,59216.985,59216.985,6.0,Active,0
2239489,2562384,378907,-41,57725.595,270000,0.0,0.000,0.0,0.000,3141.450,3375.000,3375.000,56079.270,58191.525,58191.525,7.0,Active,0
2648695,2562384,378907,-40,51462.360,270000,0.0,0.000,0.0,0.000,3089.925,8775.000,8775.000,49873.140,51920.145,51920.145,8.0,Active,0
561468,2562384,378907,-39,56570.940,270000,0.0,0.000,0.0,0.000,2761.740,0.000,0.000,53023.140,56570.940,56570.940,9.0,Active,6


In [37]:
df_credit_card_balance[df_credit_card_balance['SK_ID_PREV']==2582071].sort_values('MONTHS_BALANCE').head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD
1752153,2582071,363914,-96,35718.885,67500,0.0,0.0,0.0,0.0,3375.0,13500.0,13500.0,34088.985,35718.885,35718.885,4.0,Active,0
2198558,2582071,363914,-95,32753.070,67500,0.0,0.0,0.0,0.0,3375.0,4500.0,4500.0,31218.885,32753.070,32753.070,5.0,Active,0
3301790,2582071,363914,-94,29695.905,67500,0.0,0.0,0.0,0.0,3375.0,4500.0,4500.0,28253.070,29695.905,29695.905,6.0,Active,0
2465540,2582071,363914,-93,36326.700,67500,13500.0,13500.0,0.0,0.0,3375.0,9000.0,9000.0,34195.905,36326.700,36326.700,7.0,Active,0
985912,2582071,363914,-92,59557.770,67500,29250.0,29250.0,0.0,0.0,3375.0,9000.0,9000.0,56576.700,59557.770,59557.770,8.0,Active,0
1114122,2582071,363914,-91,52439.400,67500,0.0,0.0,0.0,0.0,3375.0,9000.0,9000.0,50557.770,52439.400,52439.400,9.0,Active,0
1386855,2582071,363914,-90,40596.300,67500,0.0,0.0,0.0,0.0,3375.0,13500.0,13500.0,38939.400,40596.300,40596.300,10.0,Active,0
1382937,2582071,363914,-89,37717.920,67500,0.0,0.0,0.0,0.0,3375.0,4500.0,4500.0,36096.300,37717.920,37717.920,11.0,Active,0
2909213,2582071,363914,-88,30255.300,67500,0.0,0.0,0.0,0.0,3375.0,9000.0,9000.0,28717.920,30255.300,30255.300,12.0,Active,0
2417567,2582071,363914,-87,22650.255,67500,0.0,0.0,0.0,0.0,3375.0,9000.0,9000.0,21255.300,22650.255,22650.255,13.0,Active,0


Neste ponto, as features 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_PAYMENT_CURRENT', 'AMT_RECEIVABLE' e 'NAME_CONTRACT_STATUS' serão removidas, por serem redundantes:

In [40]:
df_credit_card_balance.drop(['AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_PAYMENT_CURRENT', 'AMT_RECIVABLE', 'NAME_CONTRACT_STATUS'], axis='columns', inplace=True)
df_credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,SK_DPD
0,2562384,378907,-6,56.970,135000,877.5,1700.325,1800.0,0.000,0.000,35.0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,2250.0,60175.080,64875.555,69.0,0
2,1740877,371185,-7,31815.225,450000,0.0,2250.000,2250.0,26926.425,31460.085,30.0,0
3,1389973,337855,-4,236572.110,225000,2250.0,11795.760,11925.0,224949.285,233048.970,10.0,0
4,1891521,126868,-1,453919.455,450000,11547.0,22924.890,27000.0,443044.395,453919.455,101.0,0


In [44]:
df_credit_card_balance[df_credit_card_balance['SK_ID_PREV']==2562384].sort_values('MONTHS_BALANCE').head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,SK_DPD
1811769,2562384,378907,-48,69186.690,270000,67500.0,0.000,0.0,67500.000,67841.190,0.0,0
2584127,2562384,378907,-47,62727.075,270000,0.0,3498.975,4500.0,60979.410,63246.825,1.0,0
3284876,2562384,378907,-46,62577.900,270000,0.0,3399.795,3600.0,60796.035,63082.080,2.0,0
2653597,2562384,378907,-45,61484.400,270000,0.0,3346.695,3600.0,59734.170,61980.255,3.0,0
1039133,2562384,378907,-44,60736.680,270000,0.0,3291.390,3375.0,59078.160,61224.345,4.0,0
2021610,2562384,378907,-43,59786.865,270000,0.0,3242.250,3375.0,58095.090,60267.735,5.0,0
1764744,2562384,378907,-42,58743.675,270000,0.0,3194.190,3375.0,57133.755,59216.985,6.0,0
2239489,2562384,378907,-41,57725.595,270000,0.0,3141.450,3375.0,56079.270,58191.525,7.0,0
2648695,2562384,378907,-40,51462.360,270000,0.0,3089.925,8775.0,49873.140,51920.145,8.0,0
561468,2562384,378907,-39,56570.940,270000,0.0,2761.740,0.0,53023.140,56570.940,9.0,6


Será criada uma nova coluna com o saldo entre pagamento minimo e pagamento real para o mês. Será criada também uma feature auxiliar que contém valores binários para o pagamento da parcela em sua totalidade ou não (1 pagou a parcela em sua totalidade ou mais, 0 pagou menos do que o valor mínimo estipulado)

In [45]:
df_credit_card_balance['PAYMENT_BALANCE'] = df_credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT'] - df_credit_card_balance['AMT_INST_MIN_REGULARITY']
df_credit_card_balance['POSITIVE_BALANCE'] = df_credit_card_balance['PAYMENT_BALANCE'].apply(lambda x: 0 if x<0 else 1)
df_credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,CNT_INSTALMENT_MATURE_CUM,SK_DPD,PAYMENT_BALANCE,POSITIVE_BALANCE
0,2562384,378907,-6,56.970,135000,877.5,1700.325,1800.0,0.000,0.000,35.0,0,99.675,1
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,2250.0,60175.080,64875.555,69.0,0,0.000,1
2,1740877,371185,-7,31815.225,450000,0.0,2250.000,2250.0,26926.425,31460.085,30.0,0,0.000,1
3,1389973,337855,-4,236572.110,225000,2250.0,11795.760,11925.0,224949.285,233048.970,10.0,0,129.240,1
4,1891521,126868,-1,453919.455,450000,11547.0,22924.890,27000.0,443044.395,453919.455,101.0,0,4075.110,1
